In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
plt.style.use('ggplot')

## Load Data

In [ ]:
data_path = "./per_station.xlsx"
data = pd.read_excel(data_path)

# Extract the 'sc4' column data
sc_data = data['sc4'].values

In [ ]:
# Generate dummy data
series_data = sc_data[:]

# Plot the dummy data
plt.figure(figsize=(10, 6))
plt.plot(series_data, linestyle='-')
plt.title('Tsunami Simulation Data (Scenario 4)')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.grid(True)
plt.show()

## Prepare Data

In [ ]:
def prepare_data(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Define sequence length
sequence_length = 20

# Prepare the data
X, y = prepare_data(series_data, sequence_length)

# Convert to PyTorch tensors
X_tensor = torch.Tensor(X).unsqueeze(-1)  # Add an extra dimension for the input channel
y_tensor = torch.Tensor(y).unsqueeze(-1)

## LSTM Experiment

#### Build Model

In [ ]:
class StackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(StackedLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # Define a list of LSTM layers
        self.lstms = nn.ModuleList([nn.LSTM(input_size if i == 0 else hidden_size, 
                                             hidden_size, 
                                             num_layers=1,  # Each layer in the stack is a single layer LSTM
                                             batch_first=True) for i in range(num_layers)])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden and cell states for each layer
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward pass through each LSTM layer in the stack
        for i, lstm in enumerate(self.lstms):
            out, _ = lstm(x if i == 0 else out, (h0[i:i+1], c0[i:i+1]))  # Pass input or previous layer's output to the next layer
            out = self.dropout(out)  # Apply dropout
        out = self.fc(out[:, -1, :])  # Take the output of the last time step and pass it through the fully connected layer
        return out

# Define model parameters
input_size = 1
hidden_size = 32
num_layers = 2  # Set the number of layers in the LSTM stack
output_size = 1

# Instantiate the model
model = StackedLSTM(input_size, hidden_size, num_layers, output_size)
print(model)

#### Train Model

In [ ]:
# Define training parameters
num_epochs = 100
learning_rate = 0.001
batch_size = 32
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Store losses
losses = []

# Initialize the best loss as infinity
best_loss = float('inf')

# Training loop
for epoch in range(num_epochs):
    
    epoch_loss = 0.0
    
    # Shuffle the data and split into batches
    indices = torch.randperm(len(X_tensor))
    X_shuffled = X_tensor[indices]
    y_shuffled = y_tensor[indices]
    
    for i in range(0, len(X_tensor), batch_size):
        
        # Forward pass
        outputs = model(X_shuffled[i:i+batch_size])
        loss = criterion(outputs, y_shuffled[i:i+batch_size])
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accumulate loss for the epoch
        epoch_loss += loss.item()
    
    # Average epoch loss
    epoch_loss /= (len(X_tensor) / batch_size)
    losses.append(epoch_loss)
    
    # Save the model if this epoch's loss is the best we've seen so far
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'best_model.pth')

    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], MSE Loss: {epoch_loss:.6f}')

In [ ]:
# Plot learning curve
offset = 30

plt.figure(figsize=(10, 6))
plt.plot(range(offset+1, num_epochs+1), losses[offset:], linestyle='-')
plt.title('Training Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

#### Evaluate Model

In [ ]:
# Prepare the first sequence for prediction
first_sequence = torch.FloatTensor(series_data[:sequence_length]).unsqueeze(0).unsqueeze(-1)

# Make predictions for the entire time series
with torch.no_grad():
    future_periods = len(series_data) - sequence_length
    future_data = series_data[:sequence_length].tolist()

    for i in range(future_periods):
        pred = model(first_sequence)
        future_data.append(pred.item())
        
        # Update the first sequence for the next prediction
        # first_sequence = torch.cat((first_sequence[:, 1:, :], pred.unsqueeze(-1)), dim=1)
        first_sequence = torch.cat((first_sequence[:, 1:, :], torch.tensor([[series_data[i+sequence_length]]], dtype=torch.float32).unsqueeze(0)), dim=1)
        
# Plot the original and predicted simulation data
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(series_data)), series_data, linestyle='-', label='Simulation Data')
plt.plot(np.arange(len(series_data)), future_data, linestyle='--', color='g', label='Predicted Data')
plt.title('Tsunami Simulation Data Prediction with LSTM')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot the original and predicted simulation data
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(series_data[:3000])), series_data[:3000], linestyle='-', label='Simulation Data')
plt.plot(np.arange(len(series_data[:3000])), future_data[:3000], linestyle='--', color='g', label='Predicted Data')
plt.title('Tsunami Simulation Data Prediction with LSTM')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(series_data[sequence_length:], future_data[sequence_length:])
print("Mean Squared Error (MSE):", mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(series_data[sequence_length:], future_data[sequence_length:])
print("Mean Absolute Error (MAE):", mae)

## Test S1, S2, S3

In [ ]:
sc_data = data['sc1'].values
series_data = sc_data[:]

In [ ]:
# Prepare the first sequence for prediction
first_sequence = torch.FloatTensor(series_data[:sequence_length]).unsqueeze(0).unsqueeze(-1)

# Make predictions for the entire time series
with torch.no_grad():
    future_periods = len(series_data) - sequence_length
    future_data = series_data[:sequence_length].tolist()

    for i in range(future_periods):
        pred = model(first_sequence)
        future_data.append(pred.item())
        
        # Update the first sequence for the next prediction
        # first_sequence = torch.cat((first_sequence[:, 1:, :], pred.unsqueeze(-1)), dim=1)
        first_sequence = torch.cat((first_sequence[:, 1:, :], torch.tensor([[series_data[i+sequence_length]]], dtype=torch.float32).unsqueeze(0)), dim=1)
        
# Plot the original and predicted simulation data
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(series_data)), series_data, linestyle='-', label='Simulation Data')
plt.plot(np.arange(len(series_data)), future_data, linestyle='--', color='g', label='Predicted Data')
plt.title('Tsunami Simulation Data Prediction with LSTM (SC1)')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.legend()
plt.grid(True)
plt.show()

# Plot the original and predicted simulation data
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(series_data[:3000])), series_data[:3000], linestyle='-', label='Simulation Data')
plt.plot(np.arange(len(series_data[:3000])), future_data[:3000], linestyle='--', color='g', label='Predicted Data')
plt.title('Tsunami Simulation Data Prediction with LSTM (SC1)')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(series_data[sequence_length:], future_data[sequence_length:])
print("SC1 Mean Squared Error (MSE):", mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(series_data[sequence_length:], future_data[sequence_length:])
print("SC1 Mean Absolute Error (MAE):", mae)

In [ ]:
sc_data = data['sc2'].values
series_data = sc_data[:]

In [ ]:
# Prepare the first sequence for prediction
first_sequence = torch.FloatTensor(series_data[:sequence_length]).unsqueeze(0).unsqueeze(-1)

# Make predictions for the entire time series
with torch.no_grad():
    future_periods = len(series_data) - sequence_length
    future_data = series_data[:sequence_length].tolist()

    for i in range(future_periods):
        pred = model(first_sequence)
        future_data.append(pred.item())
        
        # Update the first sequence for the next prediction
        # first_sequence = torch.cat((first_sequence[:, 1:, :], pred.unsqueeze(-1)), dim=1)
        first_sequence = torch.cat((first_sequence[:, 1:, :], torch.tensor([[series_data[i+sequence_length]]], dtype=torch.float32).unsqueeze(0)), dim=1)
        
# Plot the original and predicted simulation data
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(series_data)), series_data, linestyle='-', label='Simulation Data')
plt.plot(np.arange(len(series_data)), future_data, linestyle='--', color='g', label='Predicted Data')
plt.title('Tsunami Simulation Data Prediction with LSTM (SC2)')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.legend()
plt.grid(True)
plt.show()

# Plot the original and predicted simulation data
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(series_data[:3000])), series_data[:3000], linestyle='-', label='Simulation Data')
plt.plot(np.arange(len(series_data[:3000])), future_data[:3000], linestyle='--', color='g', label='Predicted Data')
plt.title('Tsunami Simulation Data Prediction with LSTM (SC2)')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(series_data[sequence_length:], future_data[sequence_length:])
print("SC2 Mean Squared Error (MSE):", mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(series_data[sequence_length:], future_data[sequence_length:])
print("SC2 Mean Absolute Error (MAE):", mae)

In [ ]:
sc_data = data['sc3'].values
series_data = sc_data[:]

In [ ]:
# Prepare the first sequence for prediction
first_sequence = torch.FloatTensor(series_data[:sequence_length]).unsqueeze(0).unsqueeze(-1)

# Make predictions for the entire time series
with torch.no_grad():
    future_periods = len(series_data) - sequence_length
    future_data = series_data[:sequence_length].tolist()

    for i in range(future_periods):
        pred = model(first_sequence)
        future_data.append(pred.item())
        
        # Update the first sequence for the next prediction
        # first_sequence = torch.cat((first_sequence[:, 1:, :], pred.unsqueeze(-1)), dim=1)
        first_sequence = torch.cat((first_sequence[:, 1:, :], torch.tensor([[series_data[i+sequence_length]]], dtype=torch.float32).unsqueeze(0)), dim=1)
        
# Plot the original and predicted simulation data
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(series_data)), series_data, linestyle='-', label='Simulation Data')
plt.plot(np.arange(len(series_data)), future_data, linestyle='--', color='g', label='Predicted Data')
plt.title('Tsunami Simulation Data Prediction with LSTM (SC3)')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.legend()
plt.grid(True)
plt.show()

# Plot the original and predicted simulation data
plt.figure(figsize=(10, 6))
plt.plot(np.arange(len(series_data[:3000])), series_data[:3000], linestyle='-', label='Simulation Data')
plt.plot(np.arange(len(series_data[:3000])), future_data[:3000], linestyle='--', color='g', label='Predicted Data')
plt.title('Tsunami Simulation Data Prediction with LSTM (SC3)')
plt.xlabel('Period')
plt.ylabel('Attribute')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(series_data[sequence_length:], future_data[sequence_length:])
print("SC3 Mean Squared Error (MSE):", mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(series_data[sequence_length:], future_data[sequence_length:])
print("SC3 Mean Absolute Error (MAE):", mae)

---